# 통계분석

In [6]:
from ISLP import load_data
from scipy import stats
import statsmodels.api as sm

1.&nbsp; Urban변수에 따른 Sales의 차이가 있는지를 통계적으로 검증하기 위한 통계분석을 수행하고, 결과를 해석하시오. (데이터는 정규성을 만족한다고 가정하고 유의수준 0.05 하에서 검정)

In [2]:
carseats = load_data('Carseats')
carseats.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [3]:
sales_urban_yes = carseats[carseats['Urban'] == 'Yes']['Sales']
sales_urban_no = carseats[carseats['Urban'] == 'No']['Sales']

t_stat, p_value = stats.ttest_ind(sales_urban_yes, sales_urban_no, equal_var = True)

print(f't-statistic: {t_stat:.4f}, p-value: {p_value:.4f}')

t-statistic: -0.3077, p-value: 0.7585


2.&nbsp; Sales변수와 CompPrice, Income, Advertising, Population, Price, Age, Education 변수들 간의 피어슨 상관계수를 이용한 상관관계 분석을 수행하고 이를 해석하시오.

In [4]:
carseats.columns

Index(['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price',
       'ShelveLoc', 'Age', 'Education', 'Urban', 'US'],
      dtype='object')

In [5]:
variables = ['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'Age', 'Education']

corr_matrix = carseats[variables].corr(method = 'pearson')
corr_matrix

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
Sales,1.000000,0.064079,0.151951,0.269507,0.050471,-0.444951,-0.231815,-0.051955
CompPrice,0.064079,1.000000,-0.080653,-0.024199,-0.094707,0.584848,-0.100239,0.025197
Income,0.151951,-0.080653,1.000000,0.058995,-0.007877,-0.056698,-0.004670,-0.056855
Advertising,0.269507,-0.024199,0.058995,1.000000,0.265652,0.044537,-0.004557,-0.033594
Population,0.050471,-0.094707,-0.007877,0.265652,1.000000,-0.012144,-0.042663,-0.106378
Price,-0.444951,0.584848,-0.056698,0.044537,-0.012144,1.000000,-0.102177,0.011747
Age,-0.231815,-0.100239,-0.004670,-0.004557,-0.042663,-0.102177,1.000000,0.006488
Education,-0.051955,0.025197,-0.056855,-0.033594,-0.106378,0.011747,0.006488,1.000000


3.&nbsp; 종속변수를 Sales, 독립변수를 CompPrice, Income, Advertising, Population, Price, Age, Education으로 설정하고 후진제거법을 활용하여 회귀분석을 실시하고, 추정된 회귀식을 작성하시오.

In [8]:
X = carseats[['CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'Age', 'Education']]
y = carseats['Sales']
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

def backward_elimination(model, X, y, significance_level = 0.05):
    while True:
        p_values = model.pvalues
        max_p_value = p_values.max()
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            print(f'제거 변수: {excluded_feature}, p-value: {max_p_value:.4f}')
            X = X.drop(columns = [excluded_feature])
            model = sm.OLS(y, X).fit()
        else:
            break
    return model

final_model = backward_elimination(model, X, y)
print(final_model.summary())

제거 변수: Population, p-value: 0.8571
제거 변수: Education, p-value: 0.2873
                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.540
Model:                            OLS   Adj. R-squared:                  0.534
Method:                 Least Squares   F-statistic:                     92.62
Date:                Mon, 31 Mar 2025   Prob (F-statistic):           2.70e-64
Time:                        14:52:32   Log-Likelihood:                -826.92
No. Observations:                 400   AIC:                             1666.
Df Residuals:                     394   BIC:                             1690.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

4.&nbsp; 앞서 생성한 회귀모델에 대해 해석하시오.

Sales = 7.1092 + 0.0939 * CompPrice + 0.0131 * Income + 0.1306 * Advertising - 0.0925 * Price - 0.0450 * Age